<a href="https://colab.research.google.com/github/HafizaNoorUlSaba/Portfolios/blob/main/Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [5]:
from transformers import pipeline
import matplotlib.pyplot as plt
import pandas as pd
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, trainer
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
device= 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
model_ckpt= 'google/pegasus-cnn_dailymail'
tokenizer= AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [8]:
model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [12]:
!pip install datasets
from datasets import load_dataset


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.9 MB/s eta 0:00:00


In [14]:
dataset_samsum= load_dataset("knkarthick/samsum")

README.md:   0%|          | 0.00/4.36k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/504k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/522k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

In [15]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

In [18]:
split_lenghts= [len(dataset_samsum[split])for split in dataset_samsum]
split_lenghts

[14732, 818, 819]

In [22]:
print(f"features: {dataset_samsum['train'].column_names}")

features: ['id', 'dialogue', 'summary']


In [23]:
print('\nDialogue')
print(dataset_samsum['test'][1]["dialogue"])
print('\nSummary')
print(dataset_samsum['test'][1]["summary"])


Dialogue
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary
Eric and Rob are going to watch a stand-up on youtube.


In [27]:
dialogue=dataset_samsum['test'][0]["dialogue"]
dialogue

"Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

In [28]:
pipe= pipeline("summarization", model=model_ckpt)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [29]:
pipe_out= pipe(dialogue)
pipe_out

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


[{'summary_text': "Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him ."}]

In [31]:
print(pipe_out[0]['summary_text'].replace(" .<n>", ".\n"))

Amanda: Ask Larry Amanda: He called her last time we were at the park together.
Hannah: I'd rather you texted him.
Amanda: Just text him .


In [32]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
  ''' split the dataset into smaller batches that we can process simultaneously
  Yield successive batch-sized chunks from list_of_elements.'''
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i+batch_size]

In [34]:
from tqdm import tqdm

In [36]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size=16, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), column_text="dialogue", column_summary="summary"):
    model.to(device)
    model.eval()
    predictions = []
    references = []
    for i in tqdm(range(0, len(dataset), batch_size)):
        batch = dataset[i:i+batch_size]
        inputs = tokenizer(batch[column_text], max_length=1024, truncation=True, padding="longest", return_tensors="pt").to(device)
        with torch.no_grad():
            summaries = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], length_penalty=0.8, num_beams=8, max_length=128)
        decoded_summaries = tokenizer.batch_decode(summaries, skip_special_tokens=True)
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        predictions.extend(decoded_summaries)
        references.extend(batch[column_summary])
    result = metric.compute(predictions=predictions, references=references)
    return result



In [45]:
from evaluate import load

rouge_metric = load("rouge")


In [51]:
import torch
from tqdm import tqdm

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size=4, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), column_text="dialogue", column_summary="summary"):
    model.to(device)
    model.eval()
    predictions = []
    references = []

    for i in tqdm(range(0, len(dataset), batch_size)):
        batch = dataset[i:i+batch_size]
        inputs = tokenizer(batch[column_text], max_length=1024, truncation=True, padding="longest", return_tensors="pt").to(device)
        with torch.no_grad():
            summaries = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], length_penalty=0.8, num_beams=8, max_length=128)
        decoded_summaries = tokenizer.batch_decode(summaries, skip_special_tokens=True)
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        predictions.extend(decoded_summaries)
        references.extend(batch[column_summary])
        torch.cuda.empty_cache()

    result = metric.compute(predictions=predictions, references=references)
    return result


In [56]:
from transformers import pipeline


In [57]:
# Using a specific model checkpoint
summarizer = pipeline("summarization", model=model_ckpt)

# Or using the default model
summarizer = pipeline("summarization")


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


In [58]:
ARTICLE = """New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney's Office by Immigration and Customs Enforcement and the Department of Homeland Security's
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison. Her next court appearance is scheduled for May 18."""


In [59]:
summary = summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)
print(summary[0]['summary_text'])


 In total, Liana Barrientos has been married 10 times, nine times since 1999 . She is believed to still be married to four men, and at one time, she was married to eight men at once . She faces up to four years in prison .
